In [9]:
import pymeasure
from pymeasure.adapters import VXI11Adapter
import logging

from time import sleep, time

from pymeasure.instruments import Instrument
from pymeasure.instruments.validators import strict_discrete_set

In [10]:
class Lakeshore325(Instrument):
    
    def __init__(self, adapter, **kwargs):
        super().__init__(
            adapter,
            "Lake Shore 32 Temperature Controller",
            **kwargs
        )
    
    temperature_A = Instrument.measurement(
        "KRDG? A",
        """ Reads the temperature of the sensor A in Kelvin. """
    )
    
    temperature_B = Instrument.measurement(
        "KRDG? B",
        """ Reads the temperature of the sensor B in Kelvin. """
    )

    setpoint_1 = Instrument.control(
        "SETP? 1", "SETP 1, %g",
        """ A floating point property that controls the setpoint temperature
        in Kelvin for Loop 1. """
    )
    setpoint_2 = Instrument.control(
        "SETP? 2", "SETP 2, %g",
        """ A floating point property that controls the setpoint temperature
        in Kelvin for Loop 2. """
    )
    
    heater_range = Instrument.control(
        "RANGE?", "RANGE 1, %d",
        """ A string property that controls the heater range, which
        can take the values: off, low, medium, and high. These values
        correlate to 0, 0.5, 5 and 50 W respectively. """,
        validator=strict_discrete_set,
        values={'off': 0, 'low': 1, 'high': 2},
        map_values=True
    )
    
    
    def disable_heater(self):
        """ Turns the :attr:`~.heater_range` to :code:`off` to disable the heater. """
        self.heater_range = 'off'
        
    def wait_for_temperature(self, accuracy=0.1,
                             interval=0.1, sensor='A', setpoint=1, timeout=360,
                             should_stop=lambda: False):
        """ Blocks the program, waiting for the temperature to reach the setpoint
        within the accuracy (%), checking this each interval time in seconds.
        :param accuracy: An acceptable percentage deviation between the
                         setpoint and temperature
        :param interval: A time in seconds that controls the refresh rate
        :param sensor: The desired sensor to read, either A or B
        :param setpoint: The desired setpoint loop to read, either 1 or 2
        :param timeout: A timeout in seconds after which an exception is raised
        :param should_stop: A function that returns True if waiting should stop, by
                            default this always returns False
        """
        temperature_name = 'temperature_%s' % sensor
        setpoint_name = 'setpoint_%d' % setpoint
        # Only get the setpoint once, assuming it does not change
        setpoint_value = getattr(self, setpoint_name)

        def percent_difference(temperature):
            return abs(100 * (temperature - setpoint_value) / setpoint_value)
        t = time()
        while percent_difference(getattr(self, temperature_name)) > accuracy:
            sleep(interval)
            if (time() - t) > timeout:
                raise Exception((
                    "Timeout occurred after waiting %g seconds for "
                    "the LakeShore 325 temperature to reach %g K."
                ) % (timeout, setpoint))
            if should_stop():
                return
        print(f'Reached temperature preset: {setpoint_value}, actual value: {getattr(self, temperature_name)}')

Connect tempcon using Gateway IP address and GPIO of the instrument

In [11]:
adapter = VXI11Adapter("TCPIP::10.8.128.151::gpib0,12::INSTR")
tempcon=Lakeshore325(adapter)

Read temperatures from sensor A/B

In [12]:
tempcon.temperature_A

305.27

In [13]:
tempcon.temperature_B

0.0

Set temperature for loop 1 in Kelvin at 307 K

In [14]:
tempcon.setpoint_1 = 307

Set heater range to off/low/high : Manual: pp. 6-32

For loop 1:
* "low (1)": power = 2.5w 
* "high (2)": power = 25w 
* "off (0)"

For loop 2:
* "on (1)"
* "off (0)"

In this example, let's set tempctrl's heater range at "low" for loop 1

In [15]:
tempcon.heater_range = 'high'

Wait until temperature reaches the setpoint and gets stablized

In [16]:
tempcon.wait_for_temperature()

Reached temperature preset: 307.0, actual value: 306.7


Wait a few minutes, temperature controller will tweak the temperature by some sort of feedback mechanism. The temperature should be statblized. Now print the current temperature

In [17]:
print(tempcon.temperature_A)

307.0


After making necessary experiments, Disable heater

In [18]:
tempcon.disable_heater()

After a while, we should observe a decreasing temperature

In [19]:
print(tempcon.temperature_A)

305.5
